# Concrete vs Abstract Models - Pizza LP

In this notebook we'll examine the differences between concrete and abstract models.  The Pyomo documentation also discusses this but does not provide very good code examples to illustrate the differences: https://pyomo.readthedocs.io/en/latest/pyomo_overview/abstract_concrete.html

In [ ]:
import pandas as pd
import numpy as np
import pyomo.environ as pe

# Pizza LP as a Concrete Model<a id=1></a>

Concrete models in Pyomo are ones where the __data (parameters) are hard-coded into the model__.  This is the kind of model we have been building so far.  First, let's read in the data (parameters) that will be used in the objective function and constraints.

## Load in the Data

First, let's manually re-organize the data from the original spreadsheet model so that it's easy to read into pandas. This new workbook is `pizza-lp.xlsx`. Look at the 3 sheets first in Excel. We are spitting our previous `coef` table into `profit` and `resource`.

In [ ]:
file = pd.ExcelFile('pizza.xlsx')
profit = pd.read_excel(file, sheet_name = 'profit', index_col = 0)
resource = pd.read_excel(file, sheet_name = 'resource', index_col = 0)
rhs = pd.read_excel(file, sheet_name = 'rhs', index_col = 0)

print(profit)
print(resource)
print(rhs)

Here is the concrete model we built for the pizza problem in Week 5.  Notice that the parameters in the `coef` and `rhs` dataframes are written directly into the constraints, and we have to have one line per constraint.  In a way, this is silly, because the constraints all have the same form (sumproduct <= rhs).  If we had 10's or 100's or more constraints, this gets pretty inefficient!

In [ ]:
model = pe.ConcreteModel()

DV_indexes = ['plain', 'meat', 'veggie', 'supreme']
model.x = pe.Var(DV_indexes, domain = pe.NonNegativeReals)

# Objective function
model.obj = pe.Objective(expr = sum([profit.loc[i, 'profit']*model.x[i] for i in DV_indexes]), 
                         sense = pe.maximize)
# Constraints
model.cons_dough = pe.Constraint(expr = sum([resource.loc['dough', i]*model.x[i]
                                             for i in DV_indexes]) <= rhs.loc['dough', 'rhs'])
model.cons_sauce = pe.Constraint(expr = sum([resource.loc['sauce', i]*model.x[i] 
                                             for i in DV_indexes]) <= rhs.loc['sauce', 'rhs'])
model.cons_cheese = pe.Constraint(expr = sum([resource.loc['cheese', i]*model.x[i] 
                                              for i in DV_indexes]) <= rhs.loc['cheese', 'rhs'])
model.cons_meat = pe.Constraint(expr = sum([resource.loc['meat', i]*model.x[i] 
                                            for i in DV_indexes]) <= rhs.loc['meat', 'rhs'])
model.cons_veggie = pe.Constraint(expr = sum([resource.loc['veggies', i]*model.x[i] 
                                              for i in DV_indexes]) <= rhs.loc['veggies', 'rhs'])

model.pprint() 

In [ ]:
opt = pe.SolverFactory('glpk')
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

In [ ]:
obj_val = model.obj.expr()
print(f'optimal objective value maximum profit = ${obj_val:.2f}')

DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for var in DV:
        DV_solution.loc[DV.name, var] = DV[var].value
DV_solution

##### [Back to Top](#Top)

# Pizza LP (Functions)

In [ ]:
model = pe.ConcreteModel()

In [ ]:
# Decision variables definition
model.x = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.pprint()

In [ ]:
# We are going to construct a few constraints based on the resources that are used to make 
#  the pizzas. 
model.resource_set = pe.Set(initialize = ['dough', 'sauce', 'cheese', 'meat', 'veggies'])

In [ ]:
# Objective function
model.obj = pe.Objective(expr = sum([profit.loc[i, 'profit']*model.x[i] for i in DV_indexes]), 
                         sense = pe.maximize)

Now everything (constraints, decisions, and objective) are defined in terms of these.  We cannot make reference to DataFrames or other non-Pyomo objects.  Instead, we need to use the pyomo `Param` object to store our parameters.  We can use the helper function `df_to_dict` defined below to convert our dataframes to specially structured dictionaries.

Now everything is defined w.r.t. these `Param` objects, no reference to specific data values!  The trade-off is that now constraints and objectives use the `rule=` argument which is a function that defines the value of that model component.

In [ ]:
# Unlike the objective, there are multiple resource constraints that are naturally indexed by the
# resource index.  The function signature for `resource_rule`, therefore, should take one index
# (the resource_i index) as an argument.
# model.cons_dough = pe.Constraint(expr = sum([resource.loc['dough', i]*model.x[i]
#                                              for i in DV_indexes]) <= rhs.loc['dough', 'rhs'])
def resource_rule(model, ing):
    return sum([resource.loc[ing, idx]*model.x[idx] for idx in DV_indexes]) <= rhs.loc[ing, 'rhs'] 
model.resource_cons = pe.Constraint(model.resource_set, rule = resource_rule)
model.pprint()

## 2.1 Solving Normally<a id=2.1></a>

In [ ]:
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

In [ ]:
model.display()

In [ ]:
print('The Optimal Values')
obj_val = model.obj.expr()
print(f'optimal objective value = {obj_val}')

In [ ]:
DV = []  # create an empty list to store decision variables
for index in DV_indexes:
    DV.append(round(model.x[index].value, 3))
pd.DataFrame({'DV':DV_indexes,
             'Value':DV})